<a href="https://colab.research.google.com/github/maro96ost/BINA_Shawarma/blob/main/AutoScout_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#import csv files with all AutoScout Data
file_id = '12sfOuYpWJ6sVb03sY6mPg2ESpSeuCXbp'
direct_link = f'https://drive.google.com/uc?id={file_id}'
df = pd.read_csv(direct_link)
df

,Car,Price,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,02.2020,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,AUDI RS e-tron GT quattro,CHF 155'990.–,Neues Fahrzeug,9 km,Automatikgetriebe,Elektro,599 PS (440 kW),469 km,4147 Aesch
2,BMW 120d xDrive Sport Line,CHF 36'900.–,05.2023,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
3,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,05.2012,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
4,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,03.2016,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
...,...,...,...,...,...,...,...,...,...
158555,ALFA ROMEO 147 1.9 16V JTD Q2,CHF 3'890.–,11.2009,226'220 km,Schaltgetriebe manuell,Diesel,150 PS (110 kW),5.9 l/100 km,9113 Degersheim
158556,MERCEDES-BENZ SL 500 Automatic,CHF 17'900.–,07.2002,75'000 km,Automatikgetriebe,Benzin,306 PS (225 kW),12.1 l/100 km,8051 Zürich/ Schwamendingen
158557,SUBARU Forester 2.0D Luxury Lineatronic,CHF 14'500.–,08.2015,106'700 km,Stufenlos,Diesel,147 PS (108 kW),6.1 l/100 km,8253 Diessenhofen
158558,PEUGEOT 407 SW 2.0 HDI SR Selection,CHF 1'700.–,10.2006,152'347 km,Automat,Diesel,136 PS (100 kW),7.2 l/100 km,1713 St. Antoni


In [4]:
# Regular expression for date format MM.YYYY
date_pattern = r'^\d{2}\.\d{4}$'

# Filter for values that do NOT match the date format
non_date_values = df[~df['Date'].str.match(date_pattern)]['Date'].unique()
print("Unique non-date values in the 'Date' column:", non_date_values)

#Create new Columns Vorführmodell, Neuwagen, Occasion


Unique non-date values in the 'Date' column: ['Neues Fahrzeug' 'Vorführmodell' '2024' '2014' '2015' '2009' '2006'
 '2003' '2008' '2021' '2012' '2013']


In [5]:
# Create new columns with default 0
df['Neuwagen'] = 0
df['Vorführmodell'] = 0
df['Occasion'] = 0

# Set conditions
df.loc[df['Date'] == 'Neues Fahrzeug', 'Neuwagen'] = 1
df.loc[df['Date'] == 'Vorführmodell', 'Vorführmodell'] = 1
# Assuming "Occasion" is true when neither "Neuwagen" nor "Vorführmodell"
df['Occasion'] = ((df['Neuwagen'] == 0) & (df['Vorführmodell'] == 0)).astype(int)

# Reorder columns to insert the new columns after 'car'
col_order = df.columns.tolist()
# Move 'Neuwagen', 'Vorführmodell', 'Occasion' right after 'car'
new_cols = ['Neuwagen', 'Vorführmodell', 'Occasion']
for col in reversed(new_cols):  # Reverse to keep the order correct after insert
    col_order.insert(2, col_order.pop(col_order.index(col)))

df = df[col_order]




In [6]:
df.head()

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,0,0,1,02.2020,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,AUDI RS e-tron GT quattro,CHF 155'990.–,1,0,0,Neues Fahrzeug,9 km,Automatikgetriebe,Elektro,599 PS (440 kW),469 km,4147 Aesch
2,BMW 120d xDrive Sport Line,CHF 36'900.–,0,0,1,05.2023,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
3,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,0,0,1,05.2012,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
4,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,0,0,1,03.2016,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal


In [7]:
#correct neues Fahrzeug / Vorführmodell
df['Date'] = df['Date'].str.replace('Neues Fahrzeug', '01.2025', regex=False) # auf 07.2024 erhöhen
df['Date'] = df['Date'].str.replace('Vorführmodell', '01.2025', regex=False)

#correct date format
df['Date'] = df['Date'].apply(lambda x: '01.' + x if len(x) == 4 and x.isdigit() else x)

#Change Datatype
df['Date'] = pd.to_datetime(df['Date'], format='%m.%Y', errors='coerce')

In [8]:
#Drop car with Date is NaT
df_nat = df[pd.isna(df['Date'])]
df.drop(index=df_nat.index)


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,0,0,1,2020-02-01,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,AUDI RS e-tron GT quattro,CHF 155'990.–,1,0,0,2025-01-01,9 km,Automatikgetriebe,Elektro,599 PS (440 kW),469 km,4147 Aesch
2,BMW 120d xDrive Sport Line,CHF 36'900.–,0,0,1,2023-05-01,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
3,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,0,0,1,2012-05-01,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
4,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,0,0,1,2016-03-01,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
...,...,...,...,...,...,...,...,...,...,...,...,...
158555,ALFA ROMEO 147 1.9 16V JTD Q2,CHF 3'890.–,0,0,1,2009-11-01,226'220 km,Schaltgetriebe manuell,Diesel,150 PS (110 kW),5.9 l/100 km,9113 Degersheim
158556,MERCEDES-BENZ SL 500 Automatic,CHF 17'900.–,0,0,1,2002-07-01,75'000 km,Automatikgetriebe,Benzin,306 PS (225 kW),12.1 l/100 km,8051 Zürich/ Schwamendingen
158557,SUBARU Forester 2.0D Luxury Lineatronic,CHF 14'500.–,0,0,1,2015-08-01,106'700 km,Stufenlos,Diesel,147 PS (108 kW),6.1 l/100 km,8253 Diessenhofen
158558,PEUGEOT 407 SW 2.0 HDI SR Selection,CHF 1'700.–,0,0,1,2006-10-01,152'347 km,Automat,Diesel,136 PS (100 kW),7.2 l/100 km,1713 St. Antoni


In [9]:
df.dtypes
#158560 rows × 13 columns

Car                      object
Price                    object
Neuwagen                  int64
Vorführmodell             int64
Occasion                  int64
Date             datetime64[ns]
Km                       object
Transmission             object
Fuel                     object
PS                       object
Consumption              object
Location                 object
dtype: object

In [12]:
# Filter out the rows where 'Neuwagen' or 'Vorführmodell' equal 1
df_filtered = df[(df['Neuwagen'] != 1) & (df['Vorführmodell'] != 1)]

# Find duplicates in the filtered DataFrame
duplicates = df_filtered[df_filtered.duplicated(keep=False)]

# If you want to see these duplicates in the context of the original DataFrame:
duplicates_in_original_df = df.loc[duplicates.index]

duplicates_in_original_df


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
146,SKODA OCTAVIA 2.0 TSI RS DSG,CHF 39'980.–,0,0,1,2023-05-01,50 km,Halbautomatisches Getriebe,Benzin,245 PS,7.3 l/100 km,9542 Münchwilen
209,SKODA Octavia Combi 2.0 TDI RS 4x4 DSG,CHF 36'800.–,0,0,1,2022-05-01,31'200 km,Halbautomatisches Getriebe,Diesel,200 PS (147 kW),6.2 l/100 km,9320 Arbon
549,FIAT TIPO 1.5 Hybrid DCT Station Wagon,CHF 23'889.–,0,0,1,2023-10-01,20 km,Automatikgetriebe,Voll-Hybrid Benzin/Elektro,130 PS (96 kW),5.4 l/100 km,5610 Wohlen (AG)
824,VW E-UP ***TOP AUSSTATTUNG***,CHF 23'900.–,0,0,1,2023-11-01,20 km,Automat,Elektro,82 PS (60 kW),225 km,8404 Winterthur
2300,TOYOTA Aygo X 1.0 VVT-i Trend,CHF 18'900.–,0,0,1,2023-11-01,10 km,Schaltgetriebe manuell,Benzin,72 PS (53 kW),-,1196 Gland
...,...,...,...,...,...,...,...,...,...,...,...,...
158260,AUDI A5 Cabriolet 2.0 TFSI quattro S-tronic,CHF 14'499.–,0,0,1,2010-12-01,124'000 km,Halbautomatisches Getriebe,Benzin,211 PS (155 kW),7.7 l/100 km,3297 Leuzigen
158499,MASERATI Quattroporte 4.2 V8 DuoSelect,CHF 24'500.–,0,0,1,2004-12-01,77'000 km,Halbautomatisches Getriebe,Benzin,400 PS (295 kW),18.9 l/100 km,8505 Pfyn
158500,MASERATI Quattroporte 4.2 V8 DuoSelect,CHF 24'500.–,0,0,1,2004-12-01,77'000 km,Halbautomatisches Getriebe,Benzin,400 PS (295 kW),18.9 l/100 km,8505 Pfyn
158539,VW Golf Variant 1.4 TSI Comfortline DSG,CHF 6'900.–,0,0,1,2013-12-01,204'000 km,Halbautomatisches Getriebe,Benzin,122 PS (90 kW),5.1 l/100 km,1868 Collombey


In [13]:
# Assuming 'df' is your DataFrame

# Separate the DataFrame into two parts
df_no_special_rows = df[(df['Neuwagen'] != 1) & (df['Vorführmodell'] != 1)]
df_special_rows = df[(df['Neuwagen'] == 1) | (df['Vorführmodell'] == 1)]

# Drop duplicates from the part of the DataFrame without special rows
df_no_special_rows = df_no_special_rows.drop_duplicates(keep=False)

# Append the special rows back to the DataFrame without duplicates
df_cleaned = df_no_special_rows.append(df_special_rows, ignore_index=True)

df_cleaned


<ipython-input-13-aad3b1b600d4>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cleaned = df_no_special_rows.append(df_special_rows, ignore_index=True)


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,0,0,1,2020-02-01,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,CHF 36'900.–,0,0,1,2023-05-01,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,0,0,1,2012-05-01,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,0,0,1,2016-03-01,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,CHF 46'900.–,0,0,1,2018-06-01,99'500 km,Automat,Diesel,286 PS (210 kW),8.8 l/100 km,8153 Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,CHF 27'990.–,1,0,0,2025-01-01,25 km,Halbautomatisches Getriebe,Benzin,150 PS (110 kW),-,2052 Fontainemelon
156178,JAC E-JS1,CHF 17'689.–,1,0,0,2025-01-01,10 km,-,Elektro,61 PS (45 kW),330 km,9427 Wolfhalden
156179,ISUZU,CHF 48'200.–,1,0,0,2025-01-01,100 km,Schaltgetriebe manuell,-,-,-,9033 Untereggen
156180,"JAC JAC e-S4 65,7 kWh",CHF 29'989.–,1,0,0,2025-01-01,10 km,-,Elektro,193 PS (142 kW),410 km,9427 Wolfhalden


In [14]:
# check if df_cleaned has duplicates - Vorführwagen und Neuwagen werent cleaned yet!
# Check for duplicates in the cleaned DataFrame
duplicates_in_df_cleaned = df_cleaned.duplicated().sum()

# Print the number of duplicates
print(f"Number of duplicate rows in df_cleaned: {duplicates_in_df_cleaned}")

# If you need more information about the duplicates, such as viewing them
if duplicates_in_df_cleaned > 0:
    # Get the duplicate rows
    duplicate_rows = df_cleaned[df_cleaned.duplicated(keep=False)]
    print("Duplicate rows in df_cleaned:")
    print(duplicate_rows)


Number of duplicate rows in df_cleaned: 2906
Duplicate rows in df_cleaned:
                                                      Car         Price  \
120952  VW Tiguan 1.4TSIPHEV R-L DSG 245PS 5 Jahre Wer...  CHF 50'900.–   
120955           SKODA Octavia Combi 2.0 TDI DSG Ambition  CHF 39'990.–   
120977                        SKODA Fabia 1.0 MPI Essence  CHF 18'990.–   
120978                        SKODA Fabia 1.0 MPI Essence  CHF 18'990.–   
120979                        SKODA Fabia 1.0 MPI Essence  CHF 18'990.–   
...                                                   ...           ...   
156032      TOYOTA RAV-4 RAV4 2.5 Plug-In-Hybrid Platinum  CHF 66'200.–   
156034                   TOYOTA Yaris 1.5 VVT-i HSD Trend  CHF 28'890.–   
156039                   TOYOTA Yaris 1.5 VVT-i HSD Trend  CHF 28'890.–   
156045            SUZUKI S-Cross 1.5 Piz Sulai Top Hybrid  CHF 39'780.–   
156100            SUZUKI S-Cross 1.5 Piz Sulai Top Hybrid  CHF 39'780.–   

        Neuwagen  Vorfüh

In [17]:
df_cleaned[df_cleaned.isna().any(axis=1)]
df_cleaned = df_cleaned.dropna()

In [18]:
df_cleaned

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,CHF 44'450.–,0,0,1,2020-02-01,45'486 km,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,CHF 36'900.–,0,0,1,2023-05-01,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,CHF 18'799.–,0,0,1,2012-05-01,68'200 km,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,0,0,1,2016-03-01,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,CHF 46'900.–,0,0,1,2018-06-01,99'500 km,Automat,Diesel,286 PS (210 kW),8.8 l/100 km,8153 Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,CHF 27'990.–,1,0,0,2025-01-01,25 km,Halbautomatisches Getriebe,Benzin,150 PS (110 kW),-,2052 Fontainemelon
156178,JAC E-JS1,CHF 17'689.–,1,0,0,2025-01-01,10 km,-,Elektro,61 PS (45 kW),330 km,9427 Wolfhalden
156179,ISUZU,CHF 48'200.–,1,0,0,2025-01-01,100 km,Schaltgetriebe manuell,-,-,-,9033 Untereggen
156180,"JAC JAC e-S4 65,7 kWh",CHF 29'989.–,1,0,0,2025-01-01,10 km,-,Elektro,193 PS (142 kW),410 km,9427 Wolfhalden


In [23]:
# clean Price Column
df_cleaned['Price'] = df_cleaned['Price'].str.replace('CHF ', '', regex=False).str.replace("'", '', regex=False).str.rstrip('.–')
df_cleaned

<ipython-input-23-07043fa6777f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Price'] = df_cleaned['Price'].str.replace('CHF ', '', regex=False).str.replace("'", '', regex=False).str.rstrip('.–')


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450,0,0,1,2020-02-01,45486,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,36900,0,0,1,2023-05-01,28100,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799,0,0,1,2012-05-01,68200,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,46900,0,0,1,2018-06-01,99500,Automat,Diesel,286 PS (210 kW),8.8 l/100 km,8153 Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,27990,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150 PS (110 kW),-,2052 Fontainemelon
156178,JAC E-JS1,17689,1,0,0,2025-01-01,10,-,Elektro,61 PS (45 kW),330 km,9427 Wolfhalden
156179,ISUZU,48200,1,0,0,2025-01-01,100,Schaltgetriebe manuell,-,-,-,9033 Untereggen
156180,"JAC JAC e-S4 65,7 kWh",29989,1,0,0,2025-01-01,10,-,Elektro,193 PS (142 kW),410 km,9427 Wolfhalden


In [22]:
# Clean Km column
df_cleaned['Km'] = df_cleaned['Km'].str.replace("'", "").str.replace(" km", "", regex=False)
df_cleaned

<ipython-input-22-287186ef0489>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Km'] = df_cleaned['Km'].str.replace("'", "").str.replace(" km", "", regex=False)


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450,0,0,1,2020-02-01,45486,Automat,Benzin,224 PS (165 kW),8.9 l/100 km,8408 Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,36900,0,0,1,2023-05-01,28100,Automat,Diesel,190 PS (140 kW),5.2 l/100 km,7000 Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799,0,0,1,2012-05-01,68200,Automat,Diesel,204 PS (150 kW),6.5 l/100 km,8156 Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km,5417 Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,46900,0,0,1,2018-06-01,99500,Automat,Diesel,286 PS (210 kW),8.8 l/100 km,8153 Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,27990,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150 PS (110 kW),-,2052 Fontainemelon
156178,JAC E-JS1,17689,1,0,0,2025-01-01,10,-,Elektro,61 PS (45 kW),330 km,9427 Wolfhalden
156179,ISUZU,48200,1,0,0,2025-01-01,100,Schaltgetriebe manuell,-,-,-,9033 Untereggen
156180,"JAC JAC e-S4 65,7 kWh",29989,1,0,0,2025-01-01,10,-,Elektro,193 PS (142 kW),410 km,9427 Wolfhalden


In [24]:
# clean PS column
df_cleaned[['PS', 'kW']] = df_cleaned['PS'].str.extract(r'(\d+) PS \((\d+) kW\)')
df_cleaned

<ipython-input-24-bd2461d61b9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['PS', 'kW']] = df_cleaned['PS'].str.extract(r'(\d+) PS \((\d+) kW\)')
<ipython-input-24-bd2461d61b9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['PS', 'kW']] = df_cleaned['PS'].str.extract(r'(\d+) PS \((\d+) kW\)')


,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location,kW
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450,0,0,1,2020-02-01,45486,Automat,Benzin,224,8.9 l/100 km,8408 Winterthur (Wülflingen),165
1,BMW 120d xDrive Sport Line,36900,0,0,1,2023-05-01,28100,Automat,Diesel,190,5.2 l/100 km,7000 Chur,140
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799,0,0,1,2012-05-01,68200,Automat,Diesel,204,6.5 l/100 km,8156 Oberhasli,150
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290,6.6 l/100 km,5417 Untersiggenthal,213
4,VW Touareg 3.0 TDI R Line Tiptronic,46900,0,0,1,2018-06-01,99500,Automat,Diesel,286,8.8 l/100 km,8153 Rümlang ZH,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,27990,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150,-,2052 Fontainemelon,110
156178,JAC E-JS1,17689,1,0,0,2025-01-01,10,-,Elektro,61,330 km,9427 Wolfhalden,45
156179,ISUZU,48200,1,0,0,2025-01-01,100,Schaltgetriebe manuell,-,NaN,-,9033 Untereggen,NaN
156180,"JAC JAC e-S4 65,7 kWh",29989,1,0,0,2025-01-01,10,-,Elektro,193,410 km,9427 Wolfhalden,142


In [26]:
# Split Zipcode / Location
# Function to handle splitting safely
def split_location(row):
    parts = row['Location'].split(' ', 1)
    return pd.Series({'zip_code': parts[0], 'location_name': parts[1] if len(parts) > 1 else None})

# Apply the function to the DataFrame
df_cleaned = df_cleaned.join(df_cleaned.apply(split_location, axis=1))

print(df_cleaned[['zip_code', 'location_name']])


       zip_code            location_name
0          8408  Winterthur (Wülflingen)
1          7000                     Chur
2          8156                Oberhasli
3          5417          Untersiggenthal
4          8153               Rümlang ZH
...         ...                      ...
156177     2052            Fontainemelon
156178     9427               Wolfhalden
156179     9033               Untereggen
156180     9427               Wolfhalden
156181     1696         Vuisternens-Ogoz

[156181 rows x 2 columns]


In [27]:
df_cleaned

,Car,Price,Neuwagen,Vorführmodell,Occasion,Date,Km,Transmission,Fuel,PS,Consumption,Location,kW,zip_code,location_name
0,MERCEDES-BENZ GLB 250 4Matic AMG Line 8G-Troni...,44450,0,0,1,2020-02-01,45486,Automat,Benzin,224,8.9 l/100 km,8408 Winterthur (Wülflingen),165,8408,Winterthur (Wülflingen)
1,BMW 120d xDrive Sport Line,36900,0,0,1,2023-05-01,28100,Automat,Diesel,190,5.2 l/100 km,7000 Chur,140,7000,Chur
2,MERCEDES-BENZ GLK 250 CDI BlueEfficiency 4Mati...,18799,0,0,1,2012-05-01,68200,Automat,Diesel,204,6.5 l/100 km,8156 Oberhasli,150,8156,Oberhasli
3,SEAT Leon ST 2.0 TSI Cupra 290 DSG,21900,0,0,1,2016-03-01,86000,Halbautomatisches Getriebe,Benzin,290,6.6 l/100 km,5417 Untersiggenthal,213,5417,Untersiggenthal
4,VW Touareg 3.0 TDI R Line Tiptronic,46900,0,0,1,2018-06-01,99500,Automat,Diesel,286,8.8 l/100 km,8153 Rümlang ZH,210,8153,Rümlang ZH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156177,SKODA Fabia 1.5 TSI Style DSG,27990,1,0,0,2025-01-01,25,Halbautomatisches Getriebe,Benzin,150,-,2052 Fontainemelon,110,2052,Fontainemelon
156178,JAC E-JS1,17689,1,0,0,2025-01-01,10,-,Elektro,61,330 km,9427 Wolfhalden,45,9427,Wolfhalden
156179,ISUZU,48200,1,0,0,2025-01-01,100,Schaltgetriebe manuell,-,NaN,-,9033 Untereggen,NaN,9033,Untereggen
156180,"JAC JAC e-S4 65,7 kWh",29989,1,0,0,2025-01-01,10,-,Elektro,193,410 km,9427 Wolfhalden,142,9427,Wolfhalden


In [28]:
# Assuming 'df_cleaned' is your DataFrame and it already has the 'zip_code' column

# Check if all characters in 'zip_code' are numeric
numeric_zip_codes = df_cleaned['zip_code'].str.isnumeric()

# Find out if there are any non-numeric values
any_non_numeric = not numeric_zip_codes.all()

# Print result
print(f"Are all zip code values numeric? {'No, there are non-numeric values.' if any_non_numeric else 'Yes, all values are numeric.'}")

# Optionally, to see the non-numeric zip code values:
if any_non_numeric:
    non_numeric_zip_codes = df_cleaned[~numeric_zip_codes]
    print("Non-numeric zip code values:")
    print(non_numeric_zip_codes['zip_code'])


Are all zip code values numeric? Yes, all values are numeric.
